In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import conv_model
import plot_batch
import generate_report

In [2]:
! nvidia-smi

Sun Apr  5 14:51:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.78       Driver Version: 410.78       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  On   | 00000000:05:00.0 Off |                  N/A |
| 22%   29C    P8    16W / 250W |    115MiB / 12212MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
ls /scr1/checkpoints/ | grep pkl

1131088_20191120-074645_00768.ckpt.pkl
1131088_20191120-074645_00896.ckpt.pkl
1131088_20191120-074645_01024.ckpt.pkl
1131251_20191120-141715_00384.ckpt.pkl
1131251_20191120-141715_00768.ckpt.pkl
1131251_20191120-141715_00896.ckpt.pkl
1133317_20191122-143722_31872.ckpt.pkl
1133317_20191122-143722_36096.ckpt.pkl
1133317_20191122-143722_37120.ckpt.pkl
1133317_20191122-143722_37376.ckpt.pkl
1133317_20191122-143722_41088.ckpt.pkl
1133320_20191122-144210_00768.ckpt.pkl
1133320_20191122-144210_00896.ckpt.pkl
1133320_20191122-144210_01024.ckpt.pkl
1133320_20191122-144210_02432.ckpt.pkl
1133320_20191122-144210_02944.ckpt.pkl
1133320_20191122-144210_14720.ckpt.pkl
1133322_20191122-144458_01536.ckpt.pkl
1133322_20191122-144458_01664.ckpt.pkl
1133322_20191122-144458_09088.ckpt.pkl
1133322_20191122-144458_09472.ckpt.pkl
1133323_20191122-144537_07808.ckpt.pkl
1133323_20191122-144537_09216.ckpt.pkl
1133323_20191122-144537_11392.ckpt.pkl


In [12]:
%%time

%matplotlib widget

plotter, model = generate_report.run(
    model_id = 1133320,
    checkpoint_index = -1,
    example_count_log2 = 12
)
plotter()

found hypes ../hypes/1133320_20191122-144210.json 
found weights /scr1/checkpoints/1133320_20191122-144210_23296.ckpt
computing predictions


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(SelectionSlider(continuous_update=False, description='Condition:', layout=Layout(width='…

CPU times: user 6min 11s, sys: 31.6 s, total: 6min 43s
Wall time: 1min 10s


<function generate_report.get_diagnoses_plotter.<locals>.update(name, precision, example_index, log_scale)>

In [4]:
from initialize import *

In [8]:
H = load_hypes(model_id='1133317')

In [10]:
load_path = '/scr1/mimic/initial_data/'
sig_data, metadata = load_initial_data(load_path=load_path)
diagnosis = load_diagnosis(H['icd_codes'], metadata)
diagnosis = augment_diagnosis(diagnosis, metadata)
diagnosis = fix_diagnosis(diagnosis)
priors = (diagnosis == 1).sum() / (diagnosis != 0).sum()

In [11]:
priors

code
4019                   0.456686
41071                  0.076336
41401                  0.303647
4240                   0.064968
4241                   0.076158
42731                  0.306397
42732                  0.033037
4275                   0.049364
4280                   0.279266
42833                  0.025451
78552                  0.084039
99592                  0.108057
gender_F               0.456793
gender_M               0.612473
race_asian             0.030765
race_black             0.093943
race_hispanic          0.044140
race_white             0.845366
age_at_least_75        0.320542
height_at_least_70     0.304965
weight_at_least_100    0.192259
died                   0.161318
dtype: float64